## Covariance Calculation from Mock Cross-Correlation Measurements

This script calculates the covariance from the 80 cross-correlation measurements from the TreePM mocks. 

This rescaled by the diagonal term from the bootstrap estimates:



In [ ]:
import numpy as np
import time as time
import os
import lyafxcorr_kg as xcorr
import constants

# Set up matplotlib and use a nicer set of plot parameters
%config InlineBackend.rc = {}
import matplotlib as mpl
mpl.rc('mathtext',fontset='stixsans')
mpl.rc('figure', facecolor="white")
#matplotlib.rc_file("../../templates/matplotlibrc")
import matplotlib.pyplot as plt
#import matplotlib.colors as colors
%matplotlib inline

from astropy.cosmology import FlatLambdaCDM
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord

# Define cosmology
cosmo = constants.COSMOLOGY

# Read in bin edges
PiBin_fil = os.path.join(constants.XCORR_DIR_BASE, 'bins23_pi_0-30hMpc.txt')
SigBin_fil = os.path.join(constants.XCORR_DIR_BASE, 'bins10_sigma_0-30hMpc.txt')
PiBins0 = ascii.read(PiBin_fil)
SigBins0 = ascii.read(SigBin_fil)
PiEdges = PiBins0['pi_edges'].data
SigEdges = SigBins0['sigma_edges'].data
# Convert bin boundaries from Mpc/h to Mpc
PiEdges  = PiEdges/(len(PiEdges)*[cosmo.h])
SigEdges = SigEdges/(len(SigEdges)*[cosmo.h])

mockdir = os.path.join(constants.XCORR_DIR_BASE, 'mock')
bootstrap_dir = os.path.join(constants.XCORR_DIR_BASE, 'bootstrap')

n_gal = 10
n_abs = 3
n_vol = 63
nsamp = n_gal * n_abs * n_vol
# Initialize output array to store all the bootstrap samples
XCorrSample_3dhst = np.empty([len(SigEdges)-1, len(PiEdges)-1, nsamp])
XCorrSample_mosdef = np.empty([len(SigEdges)-1, len(PiEdges)-1, nsamp])
XCorrSample_clamato = np.empty([len(SigEdges)-1, len(PiEdges)-1, nsamp])
XCorrSample_vuds = np.empty([len(SigEdges)-1, len(PiEdges)-1, nsamp])
XCorrSample_zdeep = np.empty([len(SigEdges)-1, len(PiEdges)-1, nsamp])
XCorrSample_all = np.empty([len(SigEdges)-1, len(PiEdges)-1, nsamp])

ctr=0

# Read in different realizations and arrange into suitably-shaped array
for ivol in np.arange(1,n_vol+1):
    for igal in np.arange(0,n_gal):
        for iabs in np.arange(0,n_abs):
            lab_gal = igal*n_vol+ivol
            lab_abs = iabs*n_vol+ivol
    
            filesuffix = "g{0:03d}_a{1:03d}".format(lab_gal, lab_abs)
    
            XCorrTmp_zdeep=np.load(os.path.join(mockdir, "crosscorr/xcorrmock_zDeep_"+filesuffix+f"_{constants.DATA_VERSION}.npy"))
            XCorrSample_zdeep[:,:,ctr] = XCorrTmp_zdeep
    
            XCorrTmp_vuds=np.load(os.path.join(mockdir, "crosscorr/xcorrmock_vuds_"+filesuffix+f"_{constants.DATA_VERSION}.npy"))
            XCorrSample_vuds[:,:,ctr] = XCorrTmp_vuds
    
            XCorrTmp_clamato=np.load(os.path.join(mockdir, "crosscorr/xcorrmock_clamato_"+filesuffix+f"_{constants.DATA_VERSION}.npy"))
            XCorrSample_clamato[:,:,ctr] = XCorrTmp_clamato
    
            XCorrTmp_mosdef=np.load(os.path.join(mockdir, "crosscorr/xcorrmock_mosdef_"+filesuffix+f"_{constants.DATA_VERSION}.npy"))
            XCorrSample_mosdef[:,:,ctr] = XCorrTmp_mosdef
    
            XCorrTmp_3dhst=np.load(os.path.join(mockdir, "crosscorr/xcorrmock_3dhst_"+filesuffix+f"_{constants.DATA_VERSION}.npy"))
            XCorrSample_3dhst[:,:,ctr] = XCorrTmp_3dhst
            
            XCorrTmp_all=np.load(os.path.join(mockdir, "crosscorr/xcorrmock_all_"+filesuffix+f"_{constants.DATA_VERSION}.npy"))
            XCorrSample_all[:,:,ctr] = XCorrTmp_all
    
            ctr+=1
    
# Calculate covariances from mocks
Covar_mock_zdeep = np.cov(XCorrSample_zdeep.reshape(-1,XCorrSample_zdeep.shape[-1]))
Covar_mock_vuds = np.cov(XCorrSample_vuds.reshape(-1,XCorrSample_vuds.shape[-1]))
Covar_mock_clamato = np.cov(XCorrSample_clamato.reshape(-1,XCorrSample_clamato.shape[-1]))
Covar_mock_mosdef = np.cov(XCorrSample_mosdef.reshape(-1,XCorrSample_mosdef.shape[-1]))
Covar_mock_3dhst = np.cov(XCorrSample_3dhst.reshape(-1,XCorrSample_3dhst.shape[-1]))
Covar_mock_all = np.cov(XCorrSample_all.reshape(-1,XCorrSample_all.shape[-1]))

np.save(os.path.join(mockdir, 'covar', f'covar_rawmock{nsamp}_zDeep_{constants.DATA_VERSION}.npy'), Covar_mock_zdeep)
np.save(os.path.join(mockdir, 'covar', f'covar_rawmock{nsamp}_vuds_{constants.DATA_VERSION}.npy'), Covar_mock_vuds)
np.save(os.path.join(mockdir, 'covar', f'covar_rawmock{nsamp}_clamato_{constants.DATA_VERSION}.npy'), Covar_mock_clamato)
np.save(os.path.join(mockdir, 'covar', f'covar_rawmock{nsamp}_mosdef_{constants.DATA_VERSION}.npy'), Covar_mock_mosdef)
np.save(os.path.join(mockdir, 'covar', f'covar_rawmock{nsamp}_3dhst_{constants.DATA_VERSION}.npy'), Covar_mock_3dhst)
np.save(os.path.join(mockdir, 'covar', f'covar_rawmock{nsamp}_all_{constants.DATA_VERSION}.npy'), Covar_mock_all)

nbin1d = Covar_mock_zdeep.shape[0]

# Read in the bootstrap covariances, since we will use the diagonal terms to scale the 
# cross-terms
Covar_boot_zdeep = np.load(os.path.join(bootstrap_dir, f'covar_zDeep_n21600_{constants.DATA_VERSION}.npy'))
Covar_boot_vuds  = np.load(os.path.join(bootstrap_dir, f'covar_VUDS_n21600_{constants.DATA_VERSION}.npy'))
Covar_boot_clamato = np.load(os.path.join(bootstrap_dir, f'covar_CLAMATO_n21600_{constants.DATA_VERSION}.npy'))
Covar_boot_mosdef = np.load(os.path.join(bootstrap_dir, f'covar_MOSDEF_n21600_{constants.DATA_VERSION}.npy'))
Covar_boot_3dhst = np.load(os.path.join(bootstrap_dir, f'covar_3DHST_n21600_{constants.DATA_VERSION}.npy'))

In [ ]:
# Calculate correlation coefficients for both the mock and bootstrap covariances
corr_mock_zdeep = np.empty(Covar_mock_zdeep.shape)
corr_mock_vuds = np.empty(Covar_mock_zdeep.shape)
corr_mock_clamato = np.empty(Covar_mock_zdeep.shape)
corr_mock_mosdef = np.empty(Covar_mock_zdeep.shape)
corr_mock_3dhst = np.empty(Covar_mock_zdeep.shape)
corr_mock_all = np.empty(Covar_mock_zdeep.shape)

corr_boot_zdeep = np.empty(Covar_mock_zdeep.shape)
corr_boot_vuds = np.empty(Covar_mock_zdeep.shape)
corr_boot_clamato = np.empty(Covar_mock_zdeep.shape)
corr_boot_mosdef = np.empty(Covar_mock_zdeep.shape)
corr_boot_3dhst = np.empty(Covar_mock_zdeep.shape)

Covar_mockscaled_zdeep = np.empty(Covar_mock_zdeep.shape)
Covar_mockscaled_vuds = np.empty(Covar_mock_zdeep.shape)
Covar_mockscaled_clamato = np.empty(Covar_mock_zdeep.shape)
Covar_mockscaled_mosdef = np.empty(Covar_mock_zdeep.shape)
Covar_mockscaled_3dhst = np.empty(Covar_mock_zdeep.shape)

for ii in np.arange(0,nbin1d-1):
    for jj in np.arange(0,nbin1d-1):
        # Calculate correlation functions
        corr_mock_zdeep[ii,jj] = Covar_mock_zdeep[ii,jj]/\
        np.sqrt(Covar_mock_zdeep[ii,ii]*Covar_mock_zdeep[jj,jj])
        corr_boot_zdeep[ii,jj] = Covar_boot_zdeep[ii,jj]/\
        np.sqrt(Covar_boot_zdeep[ii,ii]*Covar_boot_zdeep[jj,jj])
        
        corr_mock_vuds[ii,jj] = Covar_mock_vuds[ii,jj]/\
        np.sqrt(Covar_mock_vuds[ii,ii]*Covar_mock_vuds[jj,jj])
        corr_boot_vuds[ii,jj] = Covar_boot_vuds[ii,jj]/\
        np.sqrt(Covar_boot_vuds[ii,ii]*Covar_boot_vuds[jj,jj])        
        
        corr_mock_clamato[ii,jj] = Covar_mock_clamato[ii,jj]/\
        np.sqrt(Covar_mock_clamato[ii,ii]*Covar_mock_clamato[jj,jj])
        corr_boot_clamato[ii,jj] = Covar_boot_clamato[ii,jj]/\
        np.sqrt(Covar_boot_clamato[ii,ii]*Covar_boot_clamato[jj,jj])        
        
        corr_mock_mosdef[ii,jj] = Covar_mock_mosdef[ii,jj]/\
        np.sqrt(Covar_mock_mosdef[ii,ii]*Covar_mock_mosdef[jj,jj])
        corr_boot_mosdef[ii,jj] = Covar_boot_mosdef[ii,jj]/\
        np.sqrt(Covar_boot_mosdef[ii,ii]*Covar_boot_mosdef[jj,jj])
        
        corr_mock_3dhst[ii,jj] = Covar_mock_3dhst[ii,jj]/\
        np.sqrt(Covar_mock_3dhst[ii,ii]*Covar_mock_3dhst[jj,jj])
        corr_boot_3dhst[ii,jj] = Covar_boot_3dhst[ii,jj]/\
        np.sqrt(Covar_boot_3dhst[ii,ii]*Covar_boot_3dhst[jj,jj])
        
        corr_mock_all[ii,jj] = Covar_mock_all[ii,jj]/\
        np.sqrt(Covar_mock_all[ii,ii]*Covar_mock_all[jj,jj])
        
        # Rescale off-diagonals of mock covariances by diagonal term of bootstrap covariances
        Covar_mockscaled_zdeep[ii,jj] = corr_mock_zdeep[ii,jj] * \
        np.sqrt(Covar_boot_zdeep[ii,ii]*Covar_boot_zdeep[jj,jj])

        Covar_mockscaled_vuds[ii,jj] = corr_mock_vuds[ii,jj] * \
        np.sqrt(Covar_boot_vuds[ii,ii]*Covar_boot_vuds[jj,jj])
        
        Covar_mockscaled_clamato[ii,jj] = corr_mock_clamato[ii,jj] * \
        np.sqrt(Covar_boot_clamato[ii,ii]*Covar_boot_clamato[jj,jj])

        Covar_mockscaled_mosdef[ii,jj] = corr_mock_mosdef[ii,jj] * \
        np.sqrt(Covar_boot_mosdef[ii,ii]*Covar_boot_mosdef[jj,jj])      
        
        Covar_mockscaled_3dhst[ii,jj] = corr_mock_3dhst[ii,jj] * \
        np.sqrt(Covar_boot_3dhst[ii,ii]*Covar_boot_3dhst[jj,jj])        
        
np.save(os.path.join(mockdir, 'covar', f'covar_mock{nsamp}_zDeep_{constants.DATA_VERSION}.npy'), Covar_mockscaled_zdeep)
np.save(os.path.join(mockdir, 'covar', f'covar_mock{nsamp}_vuds_{constants.DATA_VERSION}.npy'), Covar_mockscaled_vuds)
np.save(os.path.join(mockdir, 'covar', f'covar_mock{nsamp}_clamato_{constants.DATA_VERSION}.npy'), Covar_mockscaled_clamato)
np.save(os.path.join(mockdir, 'covar', f'covar_mock{nsamp}_mosdef_{constants.DATA_VERSION}.npy'), Covar_mockscaled_mosdef)
np.save(os.path.join(mockdir, 'covar', f'covar_mock{nsamp}_3dhst_{constants.DATA_VERSION}.npy'), Covar_mockscaled_3dhst)        

        

In [ ]:
def plot_covar_convergence(XCorrSample, n_spacing=100):
    XCorrSample = XCorrSample.reshape(-1, XCorrSample.shape[-1])
    n_data = np.around(np.logspace(np.log10(100), np.log10(XCorrSample.shape[-1]), n_spacing)).astype(int)
    
    diag_values = []
    for n in n_data:
        xcorr_subset = XCorrSample[:, :n]
        diag_values.append(np.diag(np.cov(xcorr_subset)))
    diag_values = np.array(diag_values)
    for diag_ind in range(diag_values.shape[1]):
        plt.plot(n_data, diag_values[:, diag_ind] / diag_values[-1, diag_ind], alpha=0.1, color='red')
    plt.axhline(1, color='black')
    plt.xlabel('# realizations in covariance calculation')
    plt.ylabel('Diagonal value / final diagonal value')

In [ ]:
plot_covar_convergence(XCorrSample_mosdef)
plt.show()
plot_covar_convergence(XCorrSample_zdeep)
plt.show()
plot_covar_convergence(XCorrSample_clamato)
plt.show()
plot_covar_convergence(XCorrSample_3dhst)
plt.show()
plot_covar_convergence(XCorrSample_vuds)
plt.show()

In [ ]:
def plot_covar(Covar_boot, Covar_mock, corr_boot, corr_mock, Covar_mockscaled):
    print(f'Condition # of bootstrap covar is {np.linalg.cond(Covar_boot)}')
    print(f'Condition # of raw mock covar is {np.linalg.cond(Covar_mock)}')
    print(f'Condition # of rescaled covar is {np.linalg.cond(Covar_mockscaled)}')

    
    fig, (ax1, ax2) = plt.subplots(1,2, figsize = (10,4))

    err_boot = np.sqrt(np.diagonal(Covar_boot))
    err_mock = np.sqrt(np.diagonal(Covar_mock))

    ax1.plot(np.arange(nbin1d), err_boot)
    ax2.plot(np.arange(nbin1d), err_mock)

    ax1.set_xlabel('Bin')
    ax2.set_xlabel('Bin')

    ax1.set_ylabel('Diagonal Std Dev')


    ax1.set_title('Bootstrap')
    ax2.set_title('Mock')

    plt.show()
    
    fig, (ax1, ax2) = plt.subplots(1,2, figsize = (10,4))
    
    X, Y = np.meshgrid(SigEdges, PiEdges)

    # SigMax = 35.
    # PiMin = -50.
    # PiMax = 50.

    pcm=ax1.pcolormesh(X, Y, np.diag(Covar_boot).reshape(-1, len(PiEdges) - 1).T, cmap='jet', shading='flat')
    ax1.set_aspect('equal')
    # ax1.set_xlim(np.min(X), SigMax)
    # ax1.set_ylim(PiMin, PiMax)
    ax1.set_xlabel(r'$\sigma\; (\mathrm{cMpc})$')
    ax1.set_ylabel(r'$\pi\; (\mathrm{cMpc})$')
    ax1.set_title("Raw boot variance")
    
    pcm=ax2.pcolormesh(X, Y, np.diag(Covar_mock).reshape(-1, len(PiEdges) - 1).T, cmap='jet', shading='flat')
    ax2.set_aspect('equal')
    # ax2.set_xlim(np.min(X), SigMax)
    # ax2.set_ylim(PiMin, PiMax)
    ax2.set_xlabel(r'$\sigma\; (\mathrm{cMpc})$')
    ax2.set_ylabel(r'$\pi\; (\mathrm{cMpc})$')
    ax2.set_title("Raw mock variance")
    
    # Plot the correlation matrices for comparison
    xmesh, ymesh = np.meshgrid(np.arange(nbin1d+1), np.arange(nbin1d+1))

    fig, (ax1, ax2) = plt.subplots(1,2, figsize = (10,4))

    ax1.pcolormesh(xmesh, ymesh, corr_boot, cmap='jet', vmin=0., vmax=1)
    ax1.set_aspect('equal')
    #ax1.set_xlim(0, 100)
    #ax1.set_ylim(0,100)
    ax1.set_title('Bootstrap Correlation')

    ax2.pcolormesh(xmesh, ymesh, corr_mock, cmap='jet', vmin=0., vmax=1)
    ax2.set_aspect('equal')
    #ax2.set_xlim(0, 100)
    #ax2.set_ylim(0,100)
    ax2.set_title('Mock Correlation')
    
    plt.savefig('figures/mock/covar/corrs.png')

    plt.show()
    
    # Plot the covariance matrices for comparison
    xmesh, ymesh = np.meshgrid(np.arange(nbin1d+1), np.arange(nbin1d+1))

    fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize = (12,3.5))

    max_bin = 207

    ax1.pcolormesh(xmesh, ymesh, Covar_boot, cmap='jet', vmin=0., vmax=0.005)
    ax1.set_aspect('equal')
    ax1.set_xlim(0, max_bin)
    ax1.set_ylim(0,max_bin)
    ax1.set_title('Bootstrap Covariance')

    ax2.pcolormesh(xmesh, ymesh, Covar_mock, cmap='jet', vmin=0., vmax=0.005)
    ax2.set_aspect('equal')
    ax2.set_xlim(0, max_bin)
    ax2.set_ylim(0,max_bin)
    ax2.set_title('Mock Covariance')

    ax3.pcolormesh(xmesh, ymesh, Covar_mockscaled, cmap='jet', vmin=0., vmax=0.005)
    ax3.set_aspect('equal')
    ax3.set_xlim(0, max_bin)
    ax3.set_ylim(0,max_bin)
    ax3.set_title('Mock Covariance (Rescaled)')

    plt.show()

In [ ]:
plot_covar(Covar_boot_mosdef, Covar_mock_mosdef, corr_boot_mosdef, corr_mock_mosdef, Covar_mockscaled_mosdef)

In [ ]:
plot_covar(Covar_boot_zdeep, Covar_mock_zdeep, corr_boot_zdeep, corr_mock_zdeep, Covar_mockscaled_zdeep)

In [ ]:
plot_covar(Covar_boot_clamato, Covar_mock_clamato, corr_boot_clamato, corr_mock_clamato, Covar_mockscaled_clamato)

In [ ]:
plot_covar(Covar_boot_3dhst, Covar_mock_3dhst, corr_boot_3dhst, corr_mock_3dhst, Covar_mockscaled_3dhst)

In [ ]:
plot_covar(Covar_boot_vuds, Covar_mock_vuds, corr_boot_vuds, corr_mock_vuds, Covar_mockscaled_vuds)

In [ ]:
xmesh, ymesh = np.meshgrid(np.arange(nbin1d+1), np.arange(nbin1d+1))
plt.figure(figsize=(5, 5))
plt.pcolormesh(xmesh, ymesh, Covar_mock_all, cmap='jet', vmin=0., vmax=0.005)
plt.show()
plt.figure(figsize=(5, 5))
plt.pcolormesh(xmesh, ymesh, corr_mock_all, cmap='jet', vmin=0., vmax=1)
plt.show()

In [ ]:
# Sum along parallel (LOS) direction.
def reduce_covar_parallel(covar):
    # Since last axis indexes realizations.
    XCorr = XCorrSample_all[..., 0]
    parallel_size = XCorr.shape[1]
    if covar.ndim != 4:
        if covar.shape[0] == XCorr.size:
            covar = covar.reshape(*XCorr.shape, *XCorr.shape)
        else:
            raise RuntimeError
    print(covar.shape)
    # Verified through linearity of covariance when summing that this is the correct procedure.
    return np.sum(covar, axis=(1, 3))

In [ ]:
oned_covar_mock_zdeep = reduce_covar_parallel(Covar_mock_zdeep)
oned_covar_mock_vuds = reduce_covar_parallel(Covar_mock_vuds)
oned_covar_mock_clamato = reduce_covar_parallel(Covar_mock_clamato)
oned_covar_mock_mosdef = reduce_covar_parallel(Covar_mock_mosdef)
oned_covar_mock_3dhst = reduce_covar_parallel(Covar_mock_3dhst)
oned_covar_mock_all = reduce_covar_parallel(Covar_mock_all)

# oned_covar_mock_zdeep = reduce_covar_parallel(corr_mock_zdeep)
# oned_covar_mock_vuds = reduce_covar_parallel(corr_mock_vuds)
# oned_covar_mock_clamato = reduce_covar_parallel(corr_mock_clamato)
# oned_covar_mock_mosdef = reduce_covar_parallel(corr_mock_mosdef)
# oned_covar_mock_3dhst = reduce_covar_parallel(corr_mock_3dhst)
# oned_covar_mock_all = reduce_covar_parallel(corr_mock_all)

In [ ]:
SigBinCenters = (SigEdges[1:] + SigEdges[:-1]) / 2

plt.plot(SigBinCenters, np.diag(oned_covar_mock_zdeep), label='zDeep')
plt.plot(SigBinCenters, np.diag(oned_covar_mock_vuds), label='VUDS')
plt.plot(SigBinCenters, np.diag(oned_covar_mock_clamato), label='CLAMATO')
plt.plot(SigBinCenters, np.diag(oned_covar_mock_mosdef), label='MOSDEF')
plt.plot(SigBinCenters, np.diag(oned_covar_mock_3dhst), label='3D-HST')
plt.plot(SigBinCenters, np.diag(oned_covar_mock_all), label='All')
plt.legend()

plt.xlabel('Transverse bin center [Mpc]')
plt.ylabel('Diagonal of (covariance summed across LOS dim.)')